In [1]:
import os


In [2]:
os.chdir("../")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_arch_name: str
    params_num_classes: int
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from Skin_Cancer_Classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Skin_Cancer_Classifier.utils.common import read_yaml, create_directories


In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "ISIC_2019_Training_Input_preprocessed")

        create_directories([Path(training.root_dir)])

        root_dir_train = Path(training.root_dir)
        root_dir_model = Path(prepare_base_model.root_dir)
        arch_name = self.params.ARCH_NAME
        base_model_path = root_dir_model / f"model_{arch_name}.pth"
        trained_model_path = root_dir_train / f"model_{arch_name}.pth"

        training_config = TrainingConfig(
            root_dir=root_dir_train,
            trained_model_path=trained_model_path,
            base_model_path= base_model_path,
            training_data=Path(training_data),
            params_arch_name = arch_name,
            params_num_classes= params.CLASSES,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE )
        

        return training_config

        

In [ ]:
from Skin_Cancer_Classifier.components.prepare_base_model import Baseline

In [ ]:
import torch

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = Baseline(True, self.config.params_arch_name, self.config.params_num_classes)
        self.model.load_state_dict(torch.load(self.config.base_model_path))

    def train_valid_generator(self):

        